### Plotting of Baseline

This file holds everything to plot the baseline for the report.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from spicetestlib.base import NetList
from spicetestlib.simulator import LTSpiceSimulator

NETLIST_PATH = os.path.join("SPICE",'amplifier_base.net')
SPICE_SIM_DATA_PATH = "SPICE_SIM_DATA"

SUPPLY_NODE = 'VDD'
GROUND_NODE = '0'
VDD = 3.3
VSS = 0

sim = LTSpiceSimulator(output_folder=SPICE_SIM_DATA_PATH)

In [ ]:
# transfer characteristics
# simulate and gather results for the base circuit
netlist = NetList(NETLIST_PATH)
netlist.net.add_instruction(".ac dec 30 1 1e4")
log, raw, raw_op = sim.run_now(netlist, 'baseline.net')

freq = np.array(np.abs(raw.get_trace('frequency')))
mag = np.array(np.abs(raw.get_trace('V(out)')))

mag_db = 20*np.log10(mag)

# plot the magnitude of the output voltage
plt.figure(figsize=(5, 3.5))
plt.plot(freq, mag_db)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude [dB]')
plt.xscale('log')
plt.xlim(freq.min(), freq.max())
plt.yticks(np.arange(-30, 21, 10))
plt.grid()
plt.tight_layout()
# save the figure as a PDF without any white rims
plt.savefig(os.path.join('figures', 'magnitude_output_voltage.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
#plt.savefig(os.path.join('figures', 'magnitude_output_voltage.pdf'), format='pdf')
plt.show()

In [ ]:
# plot the transfer characteristics for multiple values of R3

# list of values for R3 to sweep
percentages = [0.9, 0.75, 0.5, 0.25, 0.1]
R3_base_value = 2e5
R3_values = []
for percentage in percentages:
    R3_values.append((1-percentage) * R3_base_value)
R3_values.append(R3_base_value)
for percentage in reversed(percentages):
    R3_values.append((1+percentage) * R3_base_value)

# build tick labels for plots
tick_labels = []
for percentage in percentages:
    tick_labels.append(f"-{percentage*100:.0f}% ({((1-percentage) * R3_base_value)/1e3:.0f}$k\Omega$)")
tick_labels.append('base (200$k\Omega$)')
for percentage in reversed(percentages):
    tick_labels.append(f"+{percentage*100:.0f}% ({((1-percentage) * R3_base_value)/1e3:.0f}$k\Omega$)")

plt.figure(figsize=(5, 3.5))

for i, R3_value in enumerate(R3_values):
    netlist.net.set_component_value('R3',R3_value)
    log, raw, raw_op = sim.run_now(netlist, 'baseline.net')

    freq = np.array(np.abs(raw.get_trace('frequency')))
    mag = np.array(np.abs(raw.get_trace('V(out)')))

    mag_db = 20*np.log10(mag)

    plt.plot(freq, mag_db, label=tick_labels[i])

plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude [dB]')
plt.xscale('log')
plt.xlim(freq.min(), freq.max())
plt.yticks(np.arange(-30, 21, 10))
plt.grid()
plt.legend(ncol=2)
plt.tight_layout()
# save the figure as a PDF without any white rims
plt.savefig(os.path.join('figures', 'transfer_characteristics_R3.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
from spicetestlib.test_utilities import InverterObserver

# simulate dc characteristic of inverter
inv_netlist = NetList(os.path.join("SPICE",'observer_test.net'))
inv_netlist.net.add_instruction(f".dc V1 {VSS} {VDD} 0.01")
inv_netlist.net.add_instruction(".op")

# insert Inverter
inv = InverterObserver(netlist=inv_netlist, observer_node='OBS')
inv.inject()

log, raw, raw_op = sim.run_now(inv_netlist, 'observer_test.net')

# plot the dc characteristic of the inverter
plt.figure(figsize=(5, 3.5))
plt.plot(raw.get_trace('V(OBS)'), raw.get_trace('V(OBS_INVERTER)'))
plt.xlabel('Input Voltage [V]')
plt.ylabel('Output Voltage [V]')
plt.xlim(VSS, VDD)
plt.ylim(VSS, VDD+0.2)
plt.xticks([0, 0.5, 1, 2, 2.5, 3.0])
# find the point where the line crosses 1.65V on the y axis
crossing_index = np.where(np.diff(np.signbit(np.array(raw.get_trace('V(OBS_INVERTER)')) - 1.65)))[0]
if crossing_index.size > 0:
    crossing_x = raw.get_trace('V(OBS)')[crossing_index[0]]
    plt.axvline(x=crossing_x, color='r', linestyle='--')  # add a vertical line at the crossing point
    plt.xticks(list(plt.xticks()[0]) + [crossing_x])  # add the crossing point to the x-axis ticks
plt.grid()
plt.tight_layout()
# save the figure as a PDF without any white rims
plt.savefig(os.path.join('figures', 'dc_inverter_characteristic.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.show()